In [1]:
import sys
sys.path.append( "../daedalusmase_global_statistics/" )

from data import *
from plot import *

import glob
import ipywidgets as w
import os
from datetime import datetime

# GUI elements with global scope
style1 = {'description_width':'170px'}
layout1 = {'width':'780px'}
style2 = {'description_width':'95px'}
layout2 = {'width':'160px'}
OrbitPreviewImage = w.Image( format='png', visible=False )
OrbitPreviewImage.layout.visibility = 'hidden'
ExecutionTitle_Text = w.Text(value="", description='Execution title:', style=style1, layout=layout1)
ExecutionDescr_Text = w.Text(value="", description='Execution description:', style=style1, layout=layout1)
Warning_HTML = w.HTML( value ="", color="Red", visible=False )
tiegcmFolder_Dropdown    = w.Dropdown( options=["../../Sample_Data/tiegcm_data/"], description='TIEGCM files: ', style=style1, layout=layout1)
BinGroups_Dropdown       = w.Dropdown( options=["AAA", "AEM", "AFM", "AEE", "AED", "EEJ", "EPB", "SQ", "CF", "PCF"], description='Area of study: ', style=style1, layout=layout1)
OrbitFilesPath_Dropdown  = w.Dropdown( options=sorted(glob.glob("../../Sample_Data/orbital_data/")), description='Orbit Files Path: ', style=style1, layout=layout1)
SavedFilenames_Dropdown  = w.Dropdown( options=sorted(glob.glob("../RESULTS/*.nc")),  description='', style=style1, layout=layout1)
SavedFilenames2_Dropdown = w.Dropdown( options=sorted(glob.glob("../RESULTS/*.nc")),  description='', style=style1, layout=layout1)
Variable_DropDown         = w.Dropdown( options=["Joule Heating", "Electric Field North", "Electric Field East", "Pedersen Conductivity", "Hall Conductivity", "Convection Heating", "Wind Correction", "JH/mass", "JH/pressure" ],  description='Variable', style=style1, layout=layout1)
Plot_Variable_Checkbox            = w.Checkbox(value=True, description="Plot variable vs Magnetic Latitude, MLT, Altitude etc", style=style1, layout=layout1 )
Plot_VariableKpSeparated_Checkbox = w.Checkbox(value=True, description="Plot variable vs Magnetic Latitude, MLT, Altitude etc, separated by Kp", style=style1, layout=layout1 )
Plot_Distribution_Checkbox   = w.Checkbox(value=True, description="Plot distribution per bin", style=style1, layout=layout1 )
Plot_ColorSpreads_Checkbox     = w.Checkbox(value=True, description="Plot color-spread plots", style=style1, layout=layout1 )
Test_statistical_Checkbox   = w.Checkbox(value=True, description="Execute statistical test for the 2 data sets, tiegcm & orbit", style=style1, layout=layout1 )
RegressionOptions_Dropdown  = w.Dropdown( options=["None", "Polynomial - degree 1", "Polynomial - degree 2", "Polynomial - degree 3", "Polynomial - degree 4", "Polynomial - degree 5", "Polynomial - degree 6", "Power law", "Logarithmic", "Euler", "Maxwell"], value="Euler", description='Regression Analysis', style=style1, layout=layout1)


result_folders = next(os.walk("../RESULTS/" ))[1] 
try:
    result_folders.remove( "tmp" ) # remove tmp folder used by statistical test 
except:
    pass
for i in range(0, len(result_folders)):
    result_folders[i] = "../RESULTS/" + result_folders[i] + "/"
SavedFilenames_Dropdown.options += tuple(result_folders)
SavedFilenames2_Dropdown.options = SavedFilenames_Dropdown.options 


#################### EVENT LISTENERS ###########################
def Exec_Btn_Clicked( b ):
    CALCULATIONS_Title         = ExecutionTitle_Text.value
    CALCULATIONS_Description   = ExecutionDescr_Text.value
    CALCULATIONS_RegionName    = BinGroups_Dropdown.value
    CALCULATIONS_OrbitFilesPath = ""
    CALCULATIONS_TIEGCMfolder   = tiegcmFolder_Dropdown.value
    # do it
    startSecs = time.time()
    print( "Calculation for TIEGCM grid started.", datetime.now() )
    calc_stats_for_tiegcm(CALCULATIONS_TIEGCMfolder, "../RESULTS/")
    finishSecs = time.time()   
    CALCULATIONS_ExecutionDuration = finishSecs-startSecs
    # print info
    print( "Duration", CALCULATIONS_ExecutionDuration, "seconds." )
    print( "Calculation for TIEGCM grid finshed in " + str(CALCULATIONS_ExecutionDuration) + " seconds." )
    for B in Bins:
        B.printMe()
    print( "Please re-run the notebook and load the results in order to plot them." )

        
def Exec_Btn_alongOrbit_Clicked( b ):
    CALCULATIONS_Title          = ExecutionTitle_Text.value
    CALCULATIONS_Description    = ExecutionDescr_Text.value
    CALCULATIONS_RegionName     = BinGroups_Dropdown.value
    CALCULATIONS_OrbitFilesPath = OrbitFilesPath_Dropdown.value
    CALCULATIONS_TIEGCMfolder       = tiegcmFolder_Dropdown.value
    ResultsFilename = "../RESULTS/" + datetime.now().strftime("%Y_%m_%d_%H_%M_%S") + ".AlongOrbitResults.nc"
    if path.exists( ResultsFilename ):
        print( "File " + ResultsFilename + " already exists. Cannot continue in order to prevent overwriting useful data." )
    else:
        # calculate
        print( "Joule-Heating-per-Bin-Along-Orbit calculation started.", datetime.now() )
        print( "Reading TIEGCM files from", CALCULATIONS_TIEGCMfolder )
        print( "Results will be stored in", ResultsFilename, "\n" )
        startSecs = time.time()
        calc_stats_for_orbit( CALCULATIONS_TIEGCMfolder, CALCULATIONS_OrbitFilesPath, ResultsFilename )
        finishSecs = time.time()   
        CALCULATIONS_ExecutionDuration = finishSecs-startSecs
        # print info
        print( "Duration", CALCULATIONS_ExecutionDuration, "seconds." )
        print( "Joule-Heating-per-Bin Calculation finshed in " + str(CALCULATIONS_ExecutionDuration) + " seconds." )
        print( "RESULTS (stored in " + ResultsFilename + "):" )
        for B in Bins:
            B.printMe()
        print( "Please re-run the notebook and load the results in order to plot them." )




        
def Load_Btn_Clicked( b ):
    SELECTED_BIN = BinGroups_Dropdown.value
    # set the selected variable as user has chosen
    if Variable_DropDown.value == "Joule Heating":
        SELECTED_VARIABLE = "Ohmic"
        SELECTED_VARIABLE_units     = "W/m3"        
    elif Variable_DropDown.value == "Electric Field North":
        SELECTED_VARIABLE = "EEY_si"
        SELECTED_VARIABLE_units     = "mV/m"
    elif Variable_DropDown.value == "Electric Field East":
        SELECTED_VARIABLE = "EEX_si"
        SELECTED_VARIABLE_units     = "mV/m"
    elif Variable_DropDown.value == "Pedersen Conductivity":
        SELECTED_VARIABLE = "SIGMA_PED"
        SELECTED_VARIABLE_units     = "S/m"
    elif Variable_DropDown.value == "Hall Conductivity":
        SELECTED_VARIABLE = "SIGMA_HAL"
        SELECTED_VARIABLE_units     = "S/m"
    elif Variable_DropDown.value == "Convection Heating":
        SELECTED_VARIABLE = "Convection_heating"
        SELECTED_VARIABLE_units     = "W/m3"
    elif Variable_DropDown.value == "Wind Correction":
        SELECTED_VARIABLE = "Wind_heating"
        SELECTED_VARIABLE_units     = "W/m3"            
    elif Variable_DropDown.value == "JH/mass":
        SELECTED_VARIABLE = "JH/mass"
        SELECTED_VARIABLE_units     = "W/kg"            
    elif Variable_DropDown.value == "JH/pressure":
        SELECTED_VARIABLE = "JH/pressure"
        SELECTED_VARIABLE_units     = "sec^-1"            

    #### Load
    load_results( SavedFilenames_Dropdown.value, SELECTED_VARIABLE )
    
    #### Plot
    if Plot_Distribution_Checkbox.value == True:
        plot_distributions( SELECTED_BIN, SELECTED_VARIABLE )
    if Plot_Variable_Checkbox.value==True:
        plot_variable( SELECTED_BIN, SELECTED_VARIABLE )
    if Plot_VariableKpSeparated_Checkbox.value == True:    
        plot_variable_KpSeparated( SELECTED_BIN, SELECTED_VARIABLE )
    if Plot_ColorSpreads_Checkbox.value == True:
        plot_ColorSpread_KpSeparated( SELECTED_BIN, SELECTED_VARIABLE )
        
        
def CompareResults_Btn_Clicked( b ):
    SELECTED_BIN = BinGroups_Dropdown.value
    # set the selected variable as user has chosen
    if Variable_DropDown.value == "Joule Heating":
        SELECTED_VARIABLE = "Ohmic"
        SELECTED_VARIABLE_units     = "W/m3"        
    elif Variable_DropDown.value == "Electric Field North":
        SELECTED_VARIABLE = "EEY_si"
        SELECTED_VARIABLE_units     = "mV/m"
    elif Variable_DropDown.value == "Electric Field East":
        SELECTED_VARIABLE = "EEX_si"
        SELECTED_VARIABLE_units     = "mV/m"
    elif Variable_DropDown.value == "Pedersen Conductivity":
        SELECTED_VARIABLE = "SIGMA_PED"
        SELECTED_VARIABLE_units     = "S/m"
    elif Variable_DropDown.value == "Hall Conductivity":
        SELECTED_VARIABLE = "SIGMA_HAL"
        SELECTED_VARIABLE_units     = "S/m"
    elif Variable_DropDown.value == "Convection Heating":
        SELECTED_VARIABLE = "Convection_heating"
        SELECTED_VARIABLE_units     = "W/m3"
    elif Variable_DropDown.value == "Wind Correction":
        SELECTED_VARIABLE = "Wind_heating"
        SELECTED_VARIABLE_units     = "W/m3"            
    elif Variable_DropDown.value == "JH/mass":
        SELECTED_VARIABLE = "JH/mass"
        SELECTED_VARIABLE_units     = "W/kg"            
    elif Variable_DropDown.value == "JH/pressure":
        SELECTED_VARIABLE = "JH/pressure"
        SELECTED_VARIABLE_units     = "sec^-1"            
        
    plot_comparison( SavedFilenames_Dropdown.value, SavedFilenames2_Dropdown.value, SELECTED_VARIABLE, "Comparison" )
    plot_PDFperSubBin( SavedFilenames_Dropdown.value, SavedFilenames2_Dropdown.value, SELECTED_BIN, SELECTED_VARIABLE, SELECTED_VARIABLE_units )
    execute_stat_test( SavedFilenames_Dropdown.value, SavedFilenames2_Dropdown.value, SELECTED_BIN, SELECTED_VARIABLE )
            
################################################################

def tiegcmFolder_Dropdown_onChange(change):
    if change['type']=='change' and change['name']=='value' and len(change['new'])>0:
        return

        
def SavedFilenames_Dropdown_onChange(change):
    if change['type']=='change' and change['name']=='value' and len(change['new'])>0:
        file_size_in_Gigabytes = os.stat(change['new']).st_size / 1024 / 1024 / 1024
        if file_size_in_Gigabytes > 5:
            Warning_HTML.value= "<b><font color='red'>File size is " + '{0:.1f}'.format(file_size_in_Gigabytes) + " Gigabyte. Ploting will take several minutes.</b>" 
            Warning_HTML.visible=True
        elif file_size_in_Gigabytes > 1:
            Warning_HTML.value= "<b><font color='red'>File size is " + '{0:.1f}'.format(file_size_in_Gigabytes) + " Gigabyte. Ploting will take several seconds.</b>" 
            Warning_HTML.visible=True            
        else:
            Warning_HTML.visible=False



def MainTab_Changed( change ):
    if change['type']=='change' and change['name']=='selected_index':
        if change['new'] == 0:
            change = {'type':'change', 'name':'value', 'new':tiegcmFolder_Dropdown.value}
        else:
            change = {'type':'change', 'name':'value', 'new':SavedFilenames_Dropdown.value}
        tiegcmFolder_Dropdown_onChange( change )  
            
def createGUI():
    ## the top level visual elements
    MainPanel = w.VBox()    
    MainTab = w.Tab() 
    LoadCoveragePanel           = w.VBox()
    CalcCoveragePanel           = w.VBox()
    CalcCoverageAlongOrbitPanel = w.VBox()
    CompareResultsPanel         = w.VBox()
    ## the checkboxes which allow user to select which plots he wants to create
    PlotSelectionPanel = w.VBox()
    PlotSelectionPanel.children = [Plot_Variable_Checkbox, Plot_VariableKpSeparated_Checkbox, w.HBox([Plot_Distribution_Checkbox, RegressionOptions_Dropdown]), Plot_ColorSpreads_Checkbox ]
    ##
    MainTab.children = [ CalcCoverageAlongOrbitPanel, CalcCoveragePanel, LoadCoveragePanel, CompareResultsPanel ]
    MainTab.set_title(0, 'Calc along Orbit')
    MainTab.set_title(1, 'Calc for TIEGCMgrid')
    MainTab.set_title(2, 'Load Results')
    MainTab.set_title(3, 'Compare Results')
    MainPanel.children = [ MainTab, OrbitPreviewImage ]    
    ## 
    Exec_Btn = w.Button (description='Calculate for TIEGCM grid',tooltip="Click here to calculate",)
    Exec_Btn.style.button_color = 'MediumTurquoise'
    Exec_Btn.on_click( Exec_Btn_Clicked )
    CalcCoveragePanel.children = [tiegcmFolder_Dropdown, ExecutionTitle_Text, ExecutionDescr_Text, Exec_Btn ] # I removed PlotSelectionPanel
    ##
    ExecAlongOrbit_Btn = w.Button (description='Calc along Orbit',tooltip="Click here to calculate",)
    ExecAlongOrbit_Btn.style.button_color = 'Coral'
    ExecAlongOrbit_Btn.on_click( Exec_Btn_alongOrbit_Clicked )
    CalcCoverageAlongOrbitPanel.children = [tiegcmFolder_Dropdown, OrbitFilesPath_Dropdown, ExecutionTitle_Text, ExecutionDescr_Text, ExecAlongOrbit_Btn ] # I removed PlotSelectionPanel
    ##
    Load_Btn = w.Button (description='Load Results from',tooltip="Click here to plot",)
    Load_Btn.style.button_color = 'YellowGreen'
    Load_Btn.on_click( Load_Btn_Clicked )
    L2_horizontal = w.HBox()
    L2_horizontal.children = [Load_Btn, SavedFilenames_Dropdown]
    LoadCoveragePanel.children = [ w.HBox([Load_Btn, SavedFilenames_Dropdown]), Variable_DropDown, BinGroups_Dropdown, Warning_HTML, PlotSelectionPanel ]
    ##
    CompareResults_Btn = w.Button (description='Compare Results',tooltip="Click here to plot",)
    CompareResults_Btn.style.button_color = 'Gold'
    CompareResults_Btn.on_click( CompareResults_Btn_Clicked )
    CompareResultsPanel.children = [ w.HBox([w.HTML(value="TIEGCM results:"),SavedFilenames_Dropdown]), w.HBox([w.HTML(value="Orbit results:"),SavedFilenames2_Dropdown]),  CompareResults_Btn ]
    
    ## Assign event listeners
    tiegcmFolder_Dropdown.observe( tiegcmFolder_Dropdown_onChange )
    SavedFilenames_Dropdown.observe( SavedFilenames_Dropdown_onChange )
    MainTab.observe( MainTab_Changed )
    ## display orbit-related image
    change = {'type':'change', 'name':'value', 'new':tiegcmFolder_Dropdown.value}
    tiegcmFolder_Dropdown_onChange( change )        
    return MainPanel


display( createGUI() )
